#Сверточные нейросети и POS-теггинг

POS-теггинг - определение частей речи (снятие частеречной неоднозначности)

In [ ]:
!pip install pyconll

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report

import numpy as np

import pyconll

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset

In [ ]:
import random

import numpy as np
import torch

def init_random_seed(value=0):
  random.seed(value)
  np.random.seed(value)
  torch.manual_seed(value)
  torch.cuda.manual_seed(value)
  torch.backends.cudnn.deterministic = True

In [ ]:
init_random_seed()

##Загрузка текстов и разбиение на обучающую и тестовую подвыборки

In [ ]:
!wget -O ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/dan-zeman/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/dan-zeman/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2022-02-09 00:11:15--  https://raw.githubusercontent.com/dan-zeman/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81043533 (77M) [text/plain]
Saving to: ‘ru_syntagrus-ud-train.conllu’

ru_syntagrus-ud-tra 100%[===================>]  77.29M   208MB/s    in 0.4s    

2022-02-09 00:11:15 (208 MB/s) - ‘ru_syntagrus-ud-train.conllu’ saved [81043533/81043533]

--2022-02-09 00:11:16--  https://raw.githubusercontent.com/dan-zeman/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HT

In [ ]:
full_train = pyconll.load_from_file('ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [ ]:
for sent in full_train[:2]:
  for token in sent:
    print(token.form, token.upos)
  print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [ ]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [ ]:
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

In [ ]:
import re

TOKEN_RE = re.compile(r'[\w\d]+')

def tokenize_text_simple_regex(txt, min_token_size=4):
  txt = txt.lower()
  all_tokens = TOKEN_RE.findall(txt)
  return [token for token in all_tokens if len(token) >= min_token_size]

def tokenize_corpus(texts, tokenizer=tokenize_text_simple_regex, **tokenizer_kwargs):
  return [tokenizer(text, **tokenizer_kwargs) for text in texts]

In [ ]:
import collections

import numpy as np

def build_vocabulary(tokenized_texts, max_size=1000000, max_doc_freq=0.8, min_count=5, pad_word=None):
  word_counts = collections.defaultdict(int)
  doc_n = 0

  # посчитать количество документов, в которых употребляется каждое слово,
  # а также общее количество документов
  for txt in tokenized_texts:
    doc_n += 1
    unique_text_tokens = set(txt)
    for token in unique_text_tokens:
      word_counts[token] += 1

  # убрать слишком редкие и слишком частые слова
  word_counts = {word: cnt for word, cnt in word_counts.items() if cnt >= min_count and cnt / doc_n <= max_doc_freq}

  # отсортировать слова по убыванию частоты
  sorted_word_counts = sorted(word_counts.items(),
                              reverse=True,
                              key=lambda pair: pair[1])

  # добавим несуществующее слово с индексом 0 для удобства пакетной обработки
  if pad_word is not None:
    sorted_word_counts = [(pad_word, 0)] + sorted_word_counts

  # если у нас по-прежнему слишком много слов, оставить только max_size самых частотных
  if len(word_counts) > max_size:
    sorted_word_counts = sorted_word_counts[:max_size]

  # нумеруем слова
  word2id = {word: i for i, (word, _) in enumerate(sorted_word_counts)}

  # нормируем частоты слов
  word2freq = np.array([cnt / doc_n for _, cnt in sorted_word_counts], dtype='float32')

  return word2id, word2freq

In [ ]:
def character_tokenize(txt):
  return list(txt)

In [ ]:
train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, max_doc_freq=1.0, min_count=5, pad_word='<PAD>')
print('Количество уникальных символов', len(char_vocab))
print(list(char_vocab.items())[:10])

Количество уникальных символов 150
[('<PAD>', 0), (' ', 1), ('о', 2), ('е', 3), ('а', 4), ('т', 5), ('и', 6), ('н', 7), ('.', 8), ('с', 9)]


In [ ]:
UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}
label2id

{'<NOTAG>': 0,
 'ADJ': 1,
 'ADP': 2,
 'ADV': 3,
 'AUX': 4,
 'CCONJ': 5,
 'DET': 6,
 'INTJ': 7,
 'NOUN': 8,
 'NUM': 9,
 'PART': 10,
 'PRON': 11,
 'PROPN': 12,
 'PUNCT': 13,
 'SCONJ': 14,
 'SYM': 15,
 'VERB': 16,
 'X': 17}

In [ ]:
import torch

def pos_corpus_to_tensor(sentences, char2id, label2id, max_sent_len, max_token_len):
  inputs = torch.zeros((len(sentences), max_sent_len, max_token_len + 2), dtype=torch.long)
  targets = torch.zeros((len(sentences), max_sent_len), dtype=torch.long)

  for sent_i, sent in enumerate(sentences):
    for token_i, token in enumerate(sent):
      targets[sent_i, token_i] = label2id.get(token.upos, 0)
      for char_i, char in enumerate(token.form):
        inputs[sent_i, token_i, char_i + 1] = char2id.get(char, 0)

  return inputs, targets

In [ ]:
train_inputs, train_labels = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
train_dataset = TensorDataset(train_inputs, train_labels)

test_inputs, test_labels = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
test_dataset = TensorDataset(test_inputs, test_labels)

In [ ]:
train_inputs[1][:5]

tensor([[ 0, 39,  4, 25,  4, 11, 20,  7,  6, 13,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  2, 23, 11,  4,  9,  5,  7,  2, 22,  2,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 17, 16, 10,  4, 12, 11,  3,  7,  6, 19,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  9, 12, 19, 21,  6,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 40,  3, 15,  3,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0

In [ ]:
train_labels[1]

tensor([ 8,  1,  8,  8, 12, 12,  4,  8,  1, 13, 16,  2,  8,  3,  3, 13, 16,  2,
         8,  2,  8,  5,  3, 10, 16,  2,  8,  8,  2,  8, 13,  8, 13, 13,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0])

##Вспомогательная сверточная архитектура

In [ ]:
class StackedConv1d(nn.Module):
  def __init__(self, features_num, layers_n=1, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0):
    super().__init__()
    layers = []
    for _ in range(layers_n):
      layers.append(nn.Sequential(
          conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),
          nn.Dropout(dropout),
          nn.LeakyReLU()))
    self.layers = nn.ModuleList(layers)

  def forward(self, x):
    """x - BatchSize x FeaturesNum x SequenceLen"""
    for layer in self.layers:
      x = x + layer(x)
    return x

##Предсказание частей речи на уровне отдельных токенов

In [ ]:
class SingleTokenPOSTagger(nn.Module):
  def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
    super().__init__()
    self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
    self.backbone = StackedConv1d(embedding_size, **kwargs)
    self.global_pooling = nn.AdaptiveMaxPool1d(1)
    self.out = nn.Linear(embedding_size, labels_num)
    self.labels_num = labels_num

  def forward(self, tokens):
    """tokens - BatchSize x MaxSentencelen x MinTokenLen"""
    batch_size, max_sent_len, max_token_len = tokens.shape
    tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)

    char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
    char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen

    features = self.backbone(char_embeddings)

    global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

    logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
    logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum
    logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
    return logits

In [ ]:
single_token_model = SingleTokenPOSTagger(len(char_vocab), len(label2id), embedding_size=64, layers_n=3, kernel_size=3, dropout=0.3)
print('Количество параметров', sum(np.product(t.shape) for t in single_token_model.parameters()))

Количество параметров 47826


In [ ]:
import torch

def copy_data_to_device(data, device):
  if torch.is_tensor(data):
    return data.to(device)
  elif isinstance(data, (list, tuple)):
    return [copy_data_to_device(elem, device) for elem in data]
  raise ValueError('Недопустимый тип данных {}'.format(type(data)))

In [ ]:
import copy
import datetime
import traceback

import torch
from torch.utils.data import DataLoader

def train_eval_loop(model, train_dataset, val_dataset, criterion,
                    lr=1e-4, epoch_n=10, batch_size=32,
                    device=None, early_stopping_patience=10, l2_reg_alpha=0,
                    max_batches_per_epoch_train=10000,
                    max_batches_per_epoch_val=1000,
                    data_loader_ctor=DataLoader,
                    optimizer_ctor=None,
                    lr_scheduler_ctor=None,
                    shuffle_train=True,
                    dataloader_workers_n=0):
  """
  Цикл для обучения модели. После каждой эпохи качество модели оценивается по отложенной выборке.
  :param model: torch.nn.Module - обучаемая модель
  :param train_dataset: torch.utils.data.Dataset - данные для обучения
  :param val_dataset: torch.utils.data.Dataset - данные для оценки качества
  :param criterion: функция потерь для настройки модели
  :param lr: скорость обучения
  :param epoch_n: максимальное количество эпох
  :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
  :param device: cuda/cpu - устройство, на котором выполнять вычисления
  :param early_stopping_patience: наибольшее количество эпох, в течение которых допускается
    отсутствие улучшения модели, чтобы обучение продолжалось.
  :param l2_reg_alpha: коэффициент L2-регуляризации
  :param max_batches_per_epoch_train: максимальное количество итераций на одну эпоху обучения
  :param max_batches_per_epoch_val: максимальное количество итераций на одну эпоху валидации
  :param data_loader_ctor: функция для создания объекта, преобразующего датасет в батчи
    (по умолчанию torch.utils.data.DataLoader)
  :return: кортеж из двух элементов:
    - среднее значение функции потерь на валидации на лучшей эпохе
    - лучшая модель
  """
  if device is None:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
  device = torch.device(device)
  model.to(device)

  if optimizer_ctor is None:
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_reg_alpha)
  else:
    optimizer = optimizer_ctor(model.parameters(), lr=lr)

  if lr_scheduler_ctor is not None:
    lr_scheduler = lr_scheduler_ctor(optimizer)
  else:
    lr_scheduler = None

  train_dataloader = data_loader_ctor(train_dataset, batch_size=batch_size, shuffle=shuffle_train,
                                      num_workers=dataloader_workers_n)
  val_dataloader = data_loader_ctor(val_dataset, batch_size=batch_size, shuffle=False,
                                    num_workers=dataloader_workers_n)
  
  best_val_loss = float('inf')
  best_epoch_i = 0
  best_model = copy.deepcopy(model)

  for epoch_i in range(epoch_n):
    try:
      epoch_start = datetime.datetime.now()
      print('Эпоха {}'.format(epoch_i))

      model.train()
      mean_train_loss = 0
      train_batches_n = 0
      for batch_i, (batch_x, batch_y) in enumerate(train_dataloader):
        if batch_i > max_batches_per_epoch_train:
          break

        batch_x = copy_data_to_device(batch_x, device)
        batch_y = copy_data_to_device(batch_y, device)

        pred = model(batch_x)
        loss = criterion(pred, batch_y)

        model.zero_grad()
        loss.backward()

        optimizer.step()

        mean_train_loss += float(loss)
        train_batches_n += 1

      mean_train_loss /= train_batches_n
      print('Эпоха: {} итераций, {:0.2f} сек'.format(train_batches_n,
                                                     (datetime.datetime.now() - epoch_start).total_seconds()))
      print('Среднее значение функции потерь на обучении', mean_train_loss)

      model.eval()
      mean_val_loss = 0
      val_batches_n = 0

      with torch.no_grad():
        for batch_i, (batch_x, batch_y) in enumerate(val_dataloader):
          if batch_i > max_batches_per_epoch_val:
            break

          batch_x = copy_data_to_device(batch_x, device)
          batch_y = copy_data_to_device(batch_y, device)

          pred = model(batch_x)
          loss = criterion(pred, batch_y)

          mean_val_loss += float(loss)
          val_batches_n += 1

      mean_val_loss /= val_batches_n
      print('Среднее значение функции потерь на валидации', mean_val_loss)

      if mean_val_loss < best_val_loss:
        best_epoch_i = epoch_i
        best_val_loss = mean_val_loss
        best_model = copy.deepcopy(model)
        print('Новая лучшая модель!')
      elif epoch_i - best_epoch_i > early_stopping_patience:
        print('Модель не улучшилась за за последние {} эпох, прекращаем обучение'.format(
            early_stopping_patience))
        break

      if lr_scheduler is not None:
        lr_scheduler.step(mean_val_loss)

      print()
    except KeyboardInterrupt:
      print('Досрочно остановлено пользователем')
      break
    except Exception as ex:
      print('Ошибка при обучении: {}\n{}'.format(ex, traceback.format_exc()))
      break

  return best_val_loss, best_model

In [ ]:
(best_val_loss,
 best_single_token_model) = train_eval_loop(single_token_model,
                                            train_dataset,
                                            test_dataset,
                                            F.cross_entropy,
                                            lr=5e-3,
                                            epoch_n=10,
                                            batch_size=64,
                                            device='cuda',
                                            early_stopping_patience=5,
                                            max_batches_per_epoch_train=500,
                                            max_batches_per_epoch_val=100,
                                            lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                       factor=0.5,
                                                                                                                       verbose=True))

Эпоха 0
Досрочно остановлено пользователем


In [ ]:
torch.save(best_single_token_model.state_dict(), 'single_token_pos.pth')

In [ ]:
single_token_model.load_state_dict(torch.load('single_token_pos.pth'))

<All keys matched successfully>

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader

def predict_with_model(model, dataset, device=None, batch_size=32, num_workers=0, return_labels=False):
  """
  :param model: torch.nn.Module - обученная модель
  :param dataset: torch.utils.data.Dataset - данные для применения модели
  :param device: cuda/cpu - устройство, на котором выполнять вычисления
  :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
  :return: numpy.array размерности len(dataset) x *
  """
  if device is None:
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
  results_by_batch = []

  device = torch.device(device)
  model.to(device)
  model.eval()

  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
  labels = []
  with torch.no_grad():
    import tqdm
    for batch_x, batch_y in tqdm.tqdm(dataloader, total=len(dataset)/batch_size):
      batch_x = copy_data_to_device(batch_x, device)

      if return_labels:
        labels.append(batch_y.numpy())

      batch_pred = model(batch_x)
      results_by_batch.append(batch_pred.detach().cpu().numpy())

  if return_labels:
    return np.concatenate(results_by_batch, 0), np.concatenate(labels, 0)
  else:
    return np.concatenate(results_by_batch, 0)

In [ ]:
train_pred = predict_with_model(single_token_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(single_token_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

1526it [02:18, 11.03it/s]                               


##Предсказание частей речи на уровне предложений (с учетом контекста)

In [ ]:
class SentenceLevelPOSTagger(nn.Module):
  def __init__(self, vocab_size, labels_num, embedding_size=32, single_backbone_kwargs={}, context_backbone_kwargs={}):
    super().__init__()
    self.embedding_size = embedding_size
    self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
    self.single_token_backbone = StackedConv1d(embedding_size, **single_backbone_kwargs)
    self.context_backbone = StackedConv1d(embedding_size, **context_backbone_kwargs)
    self.global_pooling = nn.AdaptiveMaxPool1d(1)
    self.out = nn.Conv1d(embedding_size, labels_num, 1)
    self.labels_num = labels_num

  def forward(self, tokens):
    """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
    batch_size, max_sent_len, max_token_len = tokens.shape
    tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)

    char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
    char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
    char_features = self.single_token_backbone(char_embeddings)

    token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

    token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
    token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
    context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

    logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
    return logits

In [ ]:
sentence_level_model = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                              single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3),
                                              context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model.parameters()))

Количество параметров 84882


In [ ]:
(best_val_loss,
 best_sentence_level_model) = train_eval_loop(sentence_level_model,
                                              train_dataset,
                                              test_dataset,
                                              F.cross_entropy,
                                              lr=5e-3,
                                              epoch_n=10,
                                              batch_size=64,
                                              device='cuda',
                                              early_stopping_patience=5,
                                              max_batches_per_epoch_train=500,
                                              max_batches_per_epoch_val=100,
                                              lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                         factor=0.5,
                                                                                                                         verbose=True))

Эпоха 0
Эпоха: 501 итераций, 617.63 сек
Среднее значение функции потерь на обучении 28.594766746738
Среднее значение функции потерь на валидации 45.03266615914826
Новая лучшая модель!

Эпоха 1
Эпоха: 501 итераций, 617.19 сек
Среднее значение функции потерь на обучении 3925.2434413095198
Среднее значение функции потерь на валидации 2911.6879774036975

Эпоха 2
Эпоха: 501 итераций, 617.21 сек
Среднее значение функции потерь на обучении 20447.585280367
Среднее значение функции потерь на валидации 135069.1051206683

Эпоха 3
Эпоха: 501 итераций, 617.51 сек
Среднее значение функции потерь на обучении 410083.71241891215
Среднее значение функции потерь на валидации 799348.7892945545
Epoch     4: reducing learning rate of group 0 to 2.5000e-03.

Эпоха 4
Эпоха: 501 итераций, 617.47 сек
Среднее значение функции потерь на обучении 399584.1848490519
Среднее значение функции потерь на валидации 204326.24450804456

Эпоха 5
Эпоха: 501 итераций, 617.49 сек
Среднее значение функции потерь на обучении 505

In [ ]:
torch.save(best_sentence_level_model.state_dict(), 'sentence_level_pos.pth')

In [ ]:
sentence_level_model.load_state_dict(torch.load('sentence_level_pos.pth'))

<All keys matched successfully>

In [ ]:
train_pred = predict_with_model(sentence_level_model, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(sentence_level_model, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

100%|██████████| 206/205.75 [00:19<00:00, 10.81it/s]


Среднее значение функции потерь на валидации 44.708984375
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1231232
         ADJ       0.00      0.00      0.00     11222
         ADP       0.73      0.49      0.59     10585
         ADV       0.00      0.00      0.00      6165
         AUX       0.33      0.04      0.07      1106
       CCONJ       0.00      0.00      0.00      4410
         DET       0.02      0.01      0.02      3085
        INTJ       0.00      0.00      0.00        11
        NOUN       0.31      0.95      0.47     27974
         NUM       0.00      0.00      0.00      1829
        PART       0.00      0.00      0.00      3877
        PRON       0.00      0.00      0.00      5598
       PROPN       0.00      0.00      0.00      4438
       PUNCT       0.94      0.96      0.95     22694
       SCONJ       0.00      0.00      0.00      2258
         SYM       0.00      0.00      0.00        53
        VERB       0.00

##Применение полученных теггеров и сравнение

In [ ]:
import torch
from torch.utils.data import TensorDataset

class POSTagger:
  def __init__(self, model, char2id, id2label, max_sent_len, max_token_len):
    self.model = model
    self.char2id = char2id
    self.id2label = id2label
    self.max_sent_len = max_sent_len
    self.max_token_len = max_token_len

  def __call__(self, sentences):
    tokenized_corpus = tokenize_corpus(sentences, min_token_size=1)

    inputs = torch.zeros((len(sentences), self.max_sent_len, self.max_token_len + 2), dtype=torch.long)

    for sent_i, sentence in enumerate(tokenized_corpus):
      for token_i, token in enumerate(sentence):
        for char_i, char in enumerate(token):
          inputs[sent_i, token_i, char_i + 1] = self.char2id.get(char, 0)

    dataset = TensorDataset(inputs, torch.zeros(len(sentences)))
    predicted_probs = predict_with_model(self.model, dataset)  # SentenceN x TagsN x MaxSentLen
    predicted_classes = predicted_probs.argmax(1)

    result = []
    for sent_i, sent in enumerate(tokenized_corpus):
      result.append([self.id2label[cls] for cls in predicted_classes[sent_i, :len(sent)]])
    return result

In [ ]:
single_token_pos_tagger = POSTagger(single_token_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
sentence_level_pos_tagger = POSTagger(sentence_level_model, char_vocab, UNIQUE_TAGS, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)

In [ ]:
test_sentences = ['Мама мыла раму.',
                  'Косил косой косой косой.',
                  'Глокая куздра штеко будланула бокра и куздрячит бокрёнка.',
                  'Сяпала Калуша с Калушатами по напушке.',
                  'Пирожки поставлены в печь, мама любит печь.',
                  'Ведро дало течь, вода стала течь.',
                  'Три да три, будет дырка.',
                  'Три да три, будет шесть.',
                  'Сорок сорок']
test_sentences_tokenized = tokenize_corpus(test_sentences, min_token_size=1)

In [ ]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, single_token_pos_tagger(test_sentences)):
  print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
  print()

1it [00:00, 17.04it/s]                     

мама-NOUN мыла-ADP раму-NOUN

косил-NOUN косой-NOUN косой-NOUN косой-NOUN

глокая-NOUN куздра-NOUN штеко-NOUN будланула-NOUN бокра-NOUN и-NOUN куздрячит-NOUN бокрёнка-NOUN

сяпала-NOUN калуша-NOUN с-NOUN калушатами-NOUN по-NOUN напушке-NOUN

пирожки-NOUN поставлены-NOUN в-NOUN печь-NOUN мама-NOUN любит-NOUN печь-NOUN

ведро-NOUN дало-NOUN течь-NOUN вода-NOUN стала-NOUN течь-NOUN

три-NOUN да-NOUN три-NOUN будет-NOUN дырка-NOUN

три-NOUN да-NOUN три-NOUN будет-NOUN шесть-NOUN

сорок-NOUN сорок-NOUN



In [ ]:
for sent_tokens, sent_tags in zip(test_sentences_tokenized, sentence_level_pos_tagger(test_sentences)):
  print(' '.join('{}-{}'.format(tok, tag) for tok, tag in zip(sent_tokens, sent_tags)))
  print()

1it [00:00, 24.95it/s]                     

мама-NOUN мыла-NOUN раму-NOUN

косил-NOUN косой-NOUN косой-NOUN косой-NOUN

глокая-NOUN куздра-NOUN штеко-NOUN будланула-NOUN бокра-NOUN и-NOUN куздрячит-NOUN бокрёнка-NOUN

сяпала-NOUN калуша-NOUN с-ADP калушатами-NOUN по-ADP напушке-NOUN

пирожки-NOUN поставлены-NOUN в-ADP печь-NOUN мама-NOUN любит-NOUN печь-NOUN

ведро-NOUN дало-NOUN течь-NOUN вода-NOUN стала-NOUN течь-NOUN

три-DET да-NOUN три-NOUN будет-NOUN дырка-NOUN

три-DET да-NOUN три-NOUN будет-NOUN шесть-NOUN

сорок-NOUN сорок-NOUN



##Сверточный модуль своими руками

In [ ]:
class MyConv1d(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, padding=0):
    super().__init__()
    self.in_channels = in_channels
    self.out_channels = out_channels
    self.kernel_size = kernel_size
    self.padding = padding
    self.weight = nn.Parameter(torch.randn(in_channels * kernel_size, out_channels) / (in_channels * kernel_size),
                               requires_grad=True)
    self.bias = nn.Parameter(torch.zeros(out_channels), requires_grad=True)

  def forward(self, x):
    """x - BatchSize x InChannels x SequenceLen"""

    batch_size, src_channels, sequence_len = x.shape
    if self.padding > 0:
      pad = x.new_zeros(batch_size, src_channels, self.padding)
      x = torch.cat((pad, x, pad), dim=-1)
      sequence_len = x.shape[-1]

    chunks = []
    chunk_size = sequence_len - self.kernel_size + 1
    for offset in range(self.kernel_size):
      chunks.append(x[:, :, offset:offset + chunk_size])

    in_features = torch.cat(chunks, dim=1)  # BatchSize x InChannels * KernelSize x ChunkSize
    in_features = in_features.permute(0, 2, 1)  # BatchSize x ChunkSize x InChannels * KernelSize
    out_features = torch.bmm(in_features, self.weight.unsqueeze(0).expand(batch_size, -1, -1)) + self.bias.unsqueeze(0).unsqueeze(0)
    out_features = out_features.permute(0, 2, 1)  # BatchSize x OutChannels x ChunkSize
    return out_features

In [ ]:
sentence_level_model_my_conv = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=64,
                                                      single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d),
                                                      context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.3, conv_layer=MyConv1d))
print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model_my_conv.parameters()))

Количество параметров 84882


In [ ]:
(best_val_loss,
 best_sentence_level_model_my_conv) = train_eval_loop(sentence_level_model_my_conv,
                                                      train_dataset,
                                                      test_dataset,
                                                      F.cross_entropy,
                                                      lr=5e-3,
                                                      epoch_n=10,
                                                      batch_size=64,
                                                      device='cuda',
                                                      early_stopping_patience=5,
                                                      max_batches_per_epoch_train=500,
                                                      max_batches_per_epoch_val=100,
                                                      lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                                 factor=0.5,
                                                                                                                                 verbose=True))

Эпоха 0
Эпоха: 501 итераций, 326.71 сек
Среднее значение функции потерь на обучении 84483.94761609969
Среднее значение функции потерь на валидации 163358.02978032178
Новая лучшая модель!

Эпоха 1
Эпоха: 501 итераций, 326.64 сек
Среднее значение функции потерь на обучении 509476.910491517
Среднее значение функции потерь на валидации 531235.9186262377

Эпоха 2
Эпоха: 501 итераций, 326.63 сек
Среднее значение функции потерь на обучении 816864.9846556886
Среднее значение функции потерь на валидации 366979.08570544556

Эпоха 3
Эпоха: 501 итераций, 326.52 сек
Среднее значение функции потерь на обучении 1744346.4209081836
Среднее значение функции потерь на валидации 1512973.7747524753
Epoch     4: reducing learning rate of group 0 to 2.5000e-03.

Эпоха 4
Эпоха: 501 итераций, 326.43 сек
Среднее значение функции потерь на обучении 1168953.1333582834
Среднее значение функции потерь на валидации 435234.8917079208

Эпоха 5
Эпоха: 501 итераций, 326.60 сек
Среднее значение функции потерь на обучении

In [ ]:
torch.save(best_sentence_level_model_my_conv.state_dict(), 'sentence_level_pos_my_conv.pth')

In [ ]:
best_sentence_level_model_my_conv.load_state_dict(torch.load('sentence_level_pos_my_conv.pth'))

<All keys matched successfully>

In [ ]:
train_pred = predict_with_model(best_sentence_level_model_my_conv, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(best_sentence_level_model_my_conv, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

100%|██████████| 206/205.75 [00:24<00:00,  8.29it/s]


Среднее значение функции потерь на валидации 161955.765625
              precision    recall  f1-score   support

     <NOTAG>       0.99      0.02      0.03   1231232
         ADJ       0.03      0.00      0.00     11222
         ADP       0.16      0.01      0.02     10585
         ADV       0.00      0.00      0.00      6165
         AUX       0.00      0.00      0.00      1106
       CCONJ       0.00      0.00      0.00      4410
         DET       0.04      0.11      0.06      3085
        INTJ       0.00      0.00      0.00        11
        NOUN       0.02      0.70      0.03     27974
         NUM       0.00      0.00      0.00      1829
        PART       0.00      0.00      0.00      3877
        PRON       0.00      0.00      0.00      5598
       PROPN       0.20      0.00      0.00      4438
       PUNCT       0.50      0.97      0.66     22694
       SCONJ       0.09      0.01      0.03      2258
         SYM       0.00      0.00      0.00        53
        VERB       0.3